<a href="https://colab.research.google.com/github/guillametlucia/NMD_scorer/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import gc
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# If using CUDA
from torch.amp import autocast, GradScaler

# Import model class from NMDscorer.py
from NMDscorer import NMDscorer



Data

In [ ]:
def load_data(file_path):
    dataset = pd.read_csv(file_path)
    # One-hot encode the sequences
    dataset['EnsembleSequence'] = dataset['EnsembleSequence'].apply(one_hot_encode)
    return dataset

class PadToLength(nn.Module):
    def __init__(self, target_length: int):
        super().__init__()
        self.target_length = target_length

    def forward(self, x: torch.Tensor):
        seq_len, target_len = x.shape[-1], self.target_length

        if seq_len < target_len:
            # Pad the sequence to the target length
            padding = target_len - seq_len
            left_pad = padding // 2
            right_pad = padding - left_pad
            x = F.pad(x, (left_pad, right_pad), value=0)
            mask = torch.zeros(seq_len, dtype=torch.float32)
            mask = F.pad(mask, (left_pad, right_pad), value=1.0)
            return x, mask
        else:
            # If no padding is needed, return the original tensor and a mask of ones
            mask = torch.ones(x.shape[:-1], dtype=torch.float32)
            return x, mask

class NMDscorerDataset(Dataset):
    """
    PyTorch Dataset for NMD Scorer.
    """
    def __init__(self, dataframe, target_length=20000):
        """
        Initialize the dataset with the given data.

        dataframe : pd.DataFrame

        """
        self.target_length = PadToLength(target_length)
        dataframe.loc[:, 'EnsembleSequence'] = dataframe['EnsembleSequence'].apply(lambda x: torch.tensor(x, dtype=torch.int64))
        self.X = dataframe['EnsembleSequence'].values
        self.y = torch.tensor(dataframe['NES'].values, dtype=torch.float32)
    def __len__(self):
        """
        Get the number of samples in the dataset.

        Returns
        -------
        int
            Number of samples.
        """
        return len(self.y)

    def __getitem__(self, idx):
        """
        Get the item at the given index.
        Parameters
        ----------
        idx : int
            Index of the item.

        Returns
        -------
        tuple
            Tuple containing the feature matrix and the target vector.
        """
        sequence = self.X[idx]
        # Pad sequence if needed. Create mask
        padded_sequence , mask = self.target_length(sequence)
        return padded_sequence , mask, self.y[idx]

#Train model (can add torch.profiler to investigate memory usage. Useful if hardware limitations).

#TODO : add clipval

In [ ]:


def train_scorer(model, clip_val, train_dataloader, epochs, device, max_steps = 150000):
    """
    Fit the model according to the given training data.
    Parameters
    ----------
    train_dataloader : DataLoader
        DataLoader for the training data.
    """
    model.train()

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), eps=1e-8)
    criterion = nn.MSELoss()

    def lr_lambda(step):
        if step < 5000:
            return step / 5000
        return 1.0

    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    patience = 5
    min_delta = 0.001
    best_loss = np.inf
    counter = 0
    step = 0
    early_stop = False
    scaler = GradScaler() if torch.cuda.is_available() else None

    for epoch in range(epochs):
        logging.info(f"Epoch {epoch + 1}/{epochs}")
        if step <= max_steps:
            for batch_idx, (X_train, mask_train, y_train) in enumerate(train_dataloader):
                X_train, mask_train, y_train = X_train.to(device), mask_train.to(device), y_train.to(device)
                X_train = X_train.float()

                optimizer.zero_grad()

                if scaler is not None:
                    with autocast():
                        output = model(X_train, mask_train)
                        loss = criterion(output, y_train)
                        scaler.scale(loss).backward()

                else:
                    output = model(X_train, mask_train)
                    loss = criterion(output, y_train)
                    loss.backward()

                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_val, error_if_nonfinite=True)

                if scaler is not None:
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    optimizer.step()

                scheduler.step()

                step += 1

                del X_train, mask_train, y_train, output
                gc.collect()

        best_loss = min(best_loss, loss.item())
        if epoch + 1 % 5 == 0:
            logging.info(f"Epoch {epoch + 1}/{epochs}, Step {step}, Loss: {loss.item()}")
            if loss.item() - best_loss > min_delta:
                counter +=1
                if counter >= patience:
                    early_stop = True
            else:
                counter = 0
            early_stop = False
        if early_stop:
            logging.info(f"Early stopping at Epoch {epoch + 1}/{epochs}, Step {step}, Loss: {loss.item()}, Best loss was {best_loss} at epoch {epoch - patience + 1}")
            break
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    # Save model
    torch.save(model.state_dict(), PTH)
    #PTH =  path ending in .pth to save trained model

#Model evaluation: calculate loss and save averaged attention matrices for interpretability.

In [ ]:
def predict_scorer(model, dataloader, device):
  model.eval()
  valid_loss = 0
  criterion = nn.MSELoss()
  predictions = []
  attention_matrices = []
  mask_indices = []


  with torch.no_grad():
      for batch_idx, (X_valid, mask_valid, y_valid) in enumerate(valid_dataloader):
        X_valid, mask_valid, y_valid = X_valid.to(device), mask_valid.to(device), y_valid.to(device)
        X_valid = X_valid.float()
        predict, attention_weights, indices = model(X_valid, mask_valid)
        predictions.append(predict)
        attention_matrices.append(attention_weights)
        mask_indices.append(indices)
        valid_loss += criterion(predict, y_valid).item()

        del X_valid, mask_valid, y_valid, predict
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

  print(f'Validation loss is {valid_loss/len(valid_dataloader)}')

  # Save predictions attention matrices and mask indices, to then visualize the attention weights

  predictions = torch.cat(predictions, dim=0)
  attention_matrices = torch.cat(attention_matrices, dim=0)
  mask_indices = torch.cat(mask_indices, dim=0)

  # Save the tensors
  #test_path = path to save results
  torch.save(predictions, f'{test_path}predictions.pt')
  torch.save(attention_matrices, f'{test_path}attention_matrices.pt')
  torch.save(mask_indices, f'{test_path}mask_indices.pt')


#Model intepretation

In [ ]:
def interpret(attention_matrices)



# weight

In [ ]:
def __main__():
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  batch_size = 128
  # Load data
  #training_path and evaluation_path
  train_dataset = load_data(training_path)
  eval_dataset = load_data(evaluation_path)
  torch_train_dataset = NMDscorerDataset(train_dataset)
  torch_eval_dataset = NMDscorerDataset(eval_dataset)
  train_dataloader = DataLoader(torch_train_dataset, batch_size=batch_size, shuffle=True)
  eval_dataloader = DataLoader(torch_eval_dataset, batch_size=batch_size, shuffle=False)

  # Initialize model
  model = NMDscorer(channels= 1152,
                        num_heads=8,
                        num_conv=6,
                        window_size=128,
                        num_transformer=11,
                        dropout_rate=0.1,
                        attention_dropout_rate=0.05,
                        positional_dropout_rate=0.03,
                        key_size=128,
                        relative_position_functions=['positional_features_exponential',
                                                      'positional_features_central_mask',
                                                      'positional_features_gamma'])



  #Check if multiple GPUs are available and wrap the model with DataParallel if so
  if torch.cuda.device_count() > 1:
      model = torch.nn.DataParallel(model)
  model = model.to(device)

  # Train
  train_scorer(model, clip_val, train_dataloader, epochs, device)


  # Load model
  checkpoint = torch.load(f'{DATABASE_PATH}trained_on_all_folds_MODELNAME.pth')
  model.load_state_dict(checkpoint)

  # Evaluate
  predict_scorer(model, eval_dataloader, device)


In [ ]:
if __name__ == "__main__":
    __main__()